<h1><center> RUDARENJE ZNAČAJKI </center></h1>

<center>
    <h1><center>Microsoft malware classification</center></h1>
    <i>Klasifikacija zlonamjernog softvera</i> 
    <br><br><br>
    PRIRODOSLOVNO MATEMATIČKI FAKULTET - MATEMATIČKI ODSJEK <br>
    Projektni prijedlog iz kolegija Strojno učenje <br>
    <b>Travanj 2020.</b> 
    <br><br>
    <i>Stanišić Matea, Škrabo Petra, Terzanović Mateja, Tolja Margarita</i>  
</center>

***

U ovoj bilježnici kratko ćemo proći kroz kategorije značajki koje su dobivene iz <b>.bytes</b> i <b>.asm</b> datoteki koje reprezentiraju <i>malware</i> preuzete sa [Kaggle](https://www.kaggle.com/c/malware-classification/data)-a. 
<br>
Podaci su rudareni po uzoru na članak: [_Novel Feature Extraction, Selection and Fusion for Effective Malware Family Classification_](https://www.researchgate.net/publication/283986464_Novel_Feature_Extraction_Selection_and_Fusion_for_Effective_Malware_Family_Classification) korišteći njihov gotovi [kod](https://github.com/ManSoSec/Microsoft-Malware-Challenge) koji su objavili, uz male promjene. 
<br><br>
Za svaku kategoriju značajki, objasnit ćemo razlog zašto se gledaju te kako ih je korišteni kod izrudario.
<br>

<i><b>Napomena:</b></i> Za pokretanje koda bilo je potrebno instalirati sljedeće <code>python</code> module:
<br>
<code> hickle <br> numba <br> mahotas </code>

<br><br>

<h2><center>Značajke dobivene iz <i>.bytes</i> datoteki </center></h2>


Primjer dvije linije iz jedne <b>.bytes</b> datoteke:
<br>
<code>00401000 56 8D 44 24 08 50 8B F1 E8 1C 1B 00 00 C7 06 08
00401010 BB 42 00 8B C6 5E C2 04 00 CC CC CC CC CC CC CC</code>

Prvi broj, npr <code> 00401000 </code> označava početak adrese desno napisane heksadecimalne reprezentacije binarnog koda u memoriji. Ostale, desne, heksadecimalne vrijednosti predstavljaju važne informacije o samom <i>malwareu</i>, npr. instrukcije u kodu.

### unigram

Po definiciji iz [Hrvatskog mrežnog rječnika](http://ihjj.hr/mreznik/page/pojmovnik/6/), <b>N-gram</b> je sekvencija određene duljine koju sačinjavaju znakovi ili riječi koje se pojavljuju unutar teksta. Kao svojstvo, najčešće se koristi pri analizi teksta. Unigram (<b>1G</b>) je jedna riječ, bigram je sekvencija od dvije riječi, trigram je sekvencija od tri riječi itd. 
<br><br>
Radi smanjenja složenosti, iz podatka smo izrudarili samo informacije o unigramima, to jest o frekvencijama svakog od $257$ <i>byte</i>-ova ( $256$ + dodatni bajt <code>??</code> koji reprezentira da taj <i>byte</i> u memoriji nije povezan sa izvršnom datotekom, odnosno da taj prostor u memoriji uopće nije inicijaliziran).
<br><br>
Ovim značajkama svrha je <b>opisati strukturu programa</b>. Detaljnije o tome možemo vidjeti u članku [What Can N-Grams Learn for Malware Detection?](https://www.edwardraff.com/publications/what_can_ngrams_learn.pdf).
Frekvencije <i>byte</i>-ova dobivene su korištenjem funkcije <code>byte_1gram</code> koja kao ulaznu varijablu prima .byte datoteku <code>byte_code</code>, a kao izlaznu varijablu vraća niz od $256$ vrijednosti koje predstavljaju broj pojavljivanja svakog od $256$ različitih <i>byte</i>-ova u datoteci.


In [1]:
def byte_1gram(byte_code):
    OneByte = [0]*16**2 # polje od 256 vrijednosti, inicijalizirano na nule
    for row in byte_code: # gledamo redak po redak byte datoteke
        codes = row[:-2].split()[1:] # polje čije su vrijednosti byteovi iz trenutnog retka
        OneByteCode = [] # polje koje će sadržavati sve byteove koji nisu '??' iz trenutnog retka
        for i in codes:
            if i != '??':
                OneByteCode += [int(i,16)]

        for i in OneByteCode:
                    OneByte[i] += 1 # povećavamo broj pojavljivanja određenoga byte-a u izlaznoj varijabli
    return OneByte

<code> rezultat: 256 značajki </code>
<br><br>

### metapodaci

Pod ovim pojmom mislimo na  <b>veličinu</b> <i>bytes</i> datoteke te <b>adresu</b> prve bit sekvence (dakle, npr. <code>00401000</code> iz početnog primjera) koju smo pretvorili u dekadski oblik radi homogenosti sa drugim značajkama.

Jasno je da se ove značajke imaju važnost u klasificiranju <i>malware</i>-a jer, po istraživanju iz 2006. godine, $97$% <i>malware</i>-a su veličine ispod 1MB jer im to omogućava brže izvršavanje na ciljanoj lokaciji. Više o tome možemo pročitati u članku [Understanding and Detecting Malware Threats Based on File Sizes](https://d3gpjj9d20n0p3.cloudfront.net/fortiguard/research/DetectingMalwareThreats.pdf). Veličine njihovih binarnih reprezentacija, naravno onda varira ovisno o familiji <i>malware</i>-a.

<br>

Značajke o metapodacima dobivene su korištenjem funkcije <code>byte_meta_data</code> koja kao ulaznu varijablu prima putanju do mjesta gdje se .byte datoteka nalazi <code>file_path</code> te samu .byte datoteku u varijabli <code>file</code>, a kao izlaznu varijablu vraća polje sa dvije gore opisane značajke u varijabli <code>meta_Data</code>.


In [3]:
import os

def byte_meta_data(file_path, file):
    meta_data = []
    
    statinfo = os.stat(file_path)
    fileSize = statinfo.st_size # veličina datoteke
    meta_data.append(fileSize)

    firstLine = file.readline().split() # početna adresa
    offset = firstLine[0]
    dec = int(offset, 16)
    meta_data.append(dec)

    return meta_data

<code> rezultat: 2 značajke </code>
<br><br>

### entropija

Entropija, po definiciji [hrvatske enciklopedije](https://www.enciklopedija.hr/natuknica.aspx?ID=18042), je termin koji označava <b>mjeru neodređenosti informacija</b>. Laički rečeno, u našem slučaju, računati ćemo <b> mjeru količine <i>nereda</i></b> u distribuciji <i>byte</i>-ova u .byte datoteci. Stoga, u .byte datotekama koje reprezentiraju <i> malware </i>, vrijednost entropije je vrlo visoka u odnosnu na normalne datoteke. 
<br><br>
Značajke koje ćemo izračunati na ovaj način koriste se za detekciju moguće prisutnosti skrivanja određenih poteza (eng. <i>obfuscation</i>). Kao takve, vrlo su bitne za klasifikaciju <i>malware</i>-a.
<br><br>

#### Računanje entropije
Prvi korak je podijeliti .bytes datoteku u $N$ prozora. Dalje se, <i>[sliding window](https://www.mathworks.com/help/dsp/ug/sliding-window-method-and-exponential-weighting-method.html)</i> metodom računaju entropije: 
<br> 

$$ e_i = - \sum_{j=1}^{m} p(j) \ log_2 p(j) $$
<br>
gdje je $p(j)$ frekvencija <i>byte</i>-a $j$ u prozoru $i$, a $m$ broj različitih <i>byte</i>-a u prozoru.

<br>

U funkciji <code>get_entropy_features</code> prikazan je način gore opisanog računanja entropija. (Jedan prozor sačinjen je od $10 000$ <i>byte</i>-ova, imamo $4$ prozora.) 
<br>

Značajke potrebne za problem klasifikacije <i>malware</i>-a dobiju se računajući 
* kvartile, mean, varijancu, median, mnajveće i najmanje vrijednosti te njihove razlike entropijskih vrijednosti
* kvartile, mean, varijancu, median, mnajveće i najmanje vrijednosti te njihove razlike niza <code>np.diff(entropije)</code> (polje razlika susejdnih entropija, tj <code>[ entropija[1]-entropija[0], entropija[2]-entropija[1], ...]</code> )
* kvartile entropijskih vrijednosti svakog bloka
* mean, varijancu, median, mnajveće i najmanje vrijednosti te njihove razlike niza <code>np.diff(entropije[window])</code> svakog prozora
* $20$ percentila entropijskih vrijednosti
* $20$ percentila razlike niza <code>np.diff(entropije)</code> 


In [5]:
import sys
from numba.decorators import jit
import numpy as np
import os, gzip

def H1(data, block_size, step, counts, ent):

    entropy = 0.0
    for i in xrange(block_size):
        counts[data[i]] += 1

    for i in xrange(counts.shape[0]):
        if counts[i] > 0:
            entropy += - counts[i] / block_size * \
                np.log2(counts[i] / block_size)

    ent[0] = entropy

    for i in xrange(1, data.shape[0] - block_size):

        dec = counts[data[i - 1]]
        inc = counts[data[i + block_size - 1]]

        counts[data[i - 1]] -= 1
        counts[data[i + block_size - 1]] += 1

        entropy -= -dec / block_size * np.log2(dec / block_size)
        if dec > 1:
            entropy += -(dec - 1) / block_size * \
                np.log2((dec - 1) / block_size)

        if inc > 0:
            entropy -= -inc / block_size * np.log2(inc / block_size)

        entropy += - (inc + 1) / block_size * np.log2((inc + 1) / block_size)

        if i % step == 0:
            ent[i / step] = (entropy)


H_numba = jit(H1, nopython=True)


def get_entropy_features(byte_data):

    corr = {str(key): key for key in range(10)}
    corrl = {'A': 10, 'B': 11, 'C': 12, 'D': 13, 'E': 14, 'F': 15, '?': 16}
    corr.update(corrl)

    block_size = 10000
    step = 100
    text = byte_data

    lines = ''.join(byte_data).split('\r\n')

    t = []
    for l in lines:
        elems = l.split(' ')
        t.extend(elems[1:])
    t = ''.join(t)

    chararray = np.array([corr[x] for x in t])

    counts = np.zeros(17, dtype=np.float)
    ent = np.zeros((chararray.shape[0] - block_size) / step + 1)
    H_numba(chararray, block_size, step, counts, ent)

    return ent

<code> rezultat: 202 značajke </code>
<br><br>

### slikovna reprezentacija _malwarea_

U [raznim istraživanjima](https://www.researchgate.net/publication/228811247_Malware_Images_Visualization_and_Automatic_Classification) ispostavilo se slikovne reprezentacije <i>malware</i>-a različitih familija, ukoliko svaki <i>byte</i> reprezentiramo kao količinu sive boje jednog piksela na slici, slijede određeni uzorak.

Na primjer, pogledajmo sljedeću fotografiju.
<img src="images/malware_image_representation_fakerean_dontovoA.png">
Gornje tri slike reprezentiraju <i>malware</i> iz familije <i>Fakerean</i>, dok donje tri <i>Dontovo.A</i>. Naravno, uočavamo uzorak. 

<br>

Značajke za ovu kategoriju dobivene su pomoću [Harlick značajki](http://murphylab.web.cmu.edu/publications/boland/boland_node26.html) korišteći funkciju <code>byte_image1</code> koja kao ulaznu varijablu prima .byte datoteku <code>byte_code</code>, a kao izlaznu varijablu vraća niz od $52$ vrijednosti koje predstavljaju dobivene <i>harlick</i> značajke.

In [7]:
def byte_make_image(byte_code):
    img_array=[]
    for row in byte_code:
        xx=row.split()
        if len(xx)!=17:
            continue
        img_array.append([int(i,16) if i!='??' else 0 for i in xx[1:] ])
    img_array = np.array(img_array)
    if img_array.shape[1]!=16:
        assert(False)
    b=int((img_array.shape[0]*16)**(0.5))
    b=2**(int(log(b)/log(2))+1)
    a=int(img_array.shape[0]*16/b)
    img_array=img_array[:a*b/16,:]
    img_array=np.reshape(img_array,(a,b))
    return img_array

def byte_image1(byte_code):
    img_feat = []
    img = byte_make_image(byte_code)
    features = mahotas.features.haralick(img)
    for i in range(len(features)):
        for j in range(len(features[0])):
            img_feat.append(features[i][j])
    return img_feat

<code> rezultat: 52 značajke </code>
<br><br>

### duljina stringova

Poznato nam je da se [ASCII znakovi u decimalnom rasponu $32 - 127$](http://facweb.cs.depaul.edu/sjost/it212/documents/ascii-pr.htm) mogu ispisati, to jest, oni reprezentiaju neki znak. Na primjer, <code>2A</code> reprezentira znak <code>*</code>. (Na [ovoj poveznici](https://www.rapidtables.com/convert/number/hex-to-ascii.html) možemo pretvoriti bilo koji <i>byte</i> u znak. Nažalost, mnogi od <i>byte</i>-ova nemaju znakovnu reprezentaciju, npr. <code>11</code>.

Ovu transformaciju <i>byte</i>-ova iskoristili smo i za dobivanje novih značajki našeg problema.

Pozivanjem funkcije <code>get_strings</code> s ulaznom varijablom <code>byte_code</code> dobili smo potrebna polja za računanje broja stringova u datoteci određenih duljina. Korišteći te podatke, izrudirali smo značajke određivajući sljedeće vrijednosti:
* broj stringova duljine $i$, gdje je $1 \leq i < 100$
* broj stringova duljine veće od $1$ i manje od $10$
* broj stringova duljine veće od $10$ i manje od $30$
* broj stringova duljine veće od $30$ i manje od $60$
* broj stringova duljine veće od $60$ i manje od $90$
* broj stringova duljine veće od $1$ i manje od $100$
* broj stringova duljine veće od $100$ i manje od $150$
* broj stringova duljine veće od $150$ i manje od $250$
* broj stringova duljine veće od $250$ i manje od $450$
* broj stringova duljine veće od $400$ i manje od $600$
* broj stringova duljine veće od $600$ i manje od $900$
* broj stringova duljine veće od $900$ i manje od $1300$
* broj stringova duljine veće od $1300$ i manje od $2000$
* broj stringova duljine veće od $2000$ i manje od $3000$
* broj stringova duljine veće od $3000$ i manje od $60000$
* ukupan duljina svih stringova
* omjer, to jest ukupna duljina stringova u odnosu na broj svih znakova u datoteci


In [20]:
def extract(all_elems_codes, out, askii_list):

    MAX_STR = out.shape[0]
    cur_num_str = 0

    i = all_elems_codes.shape[0] - 1
    state = 0

    cur_end = -1
    min_length = 4
    count_one = 0
    count_two = 0
    count_three = 0

    while i >= 1:

        if all_elems_codes[i] == 0:
            if (state == 1):
                if (cur_end - i - 1 >= min_length):
                    out[cur_num_str, 0] = i + 1
                    out[cur_num_str, 1] = cur_end
                    cur_num_str += 1
                elif (cur_end - i - 1 == 1):
                    count_one += 1
                elif (cur_end - i - 1 == 2):
                    count_two += 1
                elif (cur_end - i - 1 == 3):
                    count_three += 1
                    
            state = 1
            cur_end = i
        else:
            if binary_search_numba(askii_list, all_elems_codes[i]) == -1:
                if (state == 1):
                    state = 0

                    if (cur_end - i - 1 >= min_length):
                        out[cur_num_str, 0] = i + 1
                        out[cur_num_str, 1] = cur_end
                        cur_num_str += 1
                    elif (cur_end - i - 1 == 1):
                        count_one += 1
                    elif (cur_end - i - 1 == 2):
                        count_two += 1
                    elif (cur_end - i - 1 == 3):
                        count_three += 1
        i -= 1
        if cur_num_str == MAX_STR:
            break
    return cur_num_str, count_one, count_two, count_three


ex_numba = jit(extract, nopython=True)


def get_dict():
    d = {format(key, '02X'): key for key in range(256)}
    d['??'] = 256
    return d

def get_strings(byte_data):
    
    text = byte_data
    name = ''

    lines = ''.join(text).split('\r\n')

    
    all_elems_codes = []
    convert_dict = get_dict()

    askii_list = np.array(range(32, 127) + [13, 10])
    askii_list.sort()

    for l in lines:
        elems = l.split(' ')
        all_elems_codes.extend([convert_dict[x] for x in elems[1:]])

    all_elems_codes = np.array(all_elems_codes)
    out_ = np.zeros([15000, 2], dtype=np.int64)
    m,count_one,count_two, count_three = ex_numba(all_elems_codes, out_, askii_list)

    string_total_len = np.sum(out_[:,1] - out_[:,0]) + count_one + count_two + count_three
    string_ratio = float(string_total_len)/len(all_elems_codes)

    strings = []
    for i in range(m):
        strings.extend(
            [''.join([chr(x) for x in all_elems_codes[out_[i, 0]:out_[i, 1]]])])

    return [name, strings, [count_one,count_two,count_three,string_total_len,string_ratio]]



<code> rezultat: 116 značajki </code>
<br><br><br><br>

***

<h2><center>Značajke dobivene iz <i>.asm</i> datoteki </center></h2>

Primjer nekoliko linija iz jedne <b>.asm</b> datoteke:
<br>
<code>.text:00401016						       ; ---------------------------------------------------------------------------
.text:00401019 CC CC CC	CC CC CC CC					       align 10h
.text:00401020 C7 01 08	BB 42 00					       mov     dword ptr [ecx],	offset off_42BB08
.text:00401026 E9 26 1C	00 00						       jmp     sub_402C51
.text:00401026						       ; ---------------------------------------------------------------------------</code>

<br>
Dakle, radi se o asemblerskom kodu. Stoga će nam gotovo sve kategorije značajki dobivene iz .asm datoteke govoriti o <b>ponašanju</b> <i>malware</i>-a - npr. frekvencijom korištenja operacije <code>add</code> ili frekvencijom poziva funkcije <code>HeapCreate</code>.

### metapodaci

Ove značajke odabrane su iz istog razloga kao i značajke o metapodacima .bytes datoteka. Dobili smo ih pozivanjem funkcije <code>asm_meta_data</code> koja kao ulazne varijable prima <code>file_path</code>, odnosno putanju do mjesta gdje se datoteka nalazi, radi određivanje njene veličine, te samu .asm datoteku - <code>asm_code</code> radi određivanja početne adrese. Funkcija vraća <code>meta_data</code> u kojima šalje vrijednosti te dvije značajke.

In [10]:
def asm_meta_data(file_path, asm_code):

    meta_data = []
    statinfo = os.stat(file_path)
    fileSize = statinfo.st_size
    meta_data.append(fileSize)

    loc = 0
    for row in asm_code:
        loc += 1
    meta_data.append(loc)

    return meta_data

<code> rezultat: 2 značajke </code>
<br><br>

### simboli ( + - * ] [ ? @ )

Ova skupina značajki sastoji se od broja pojavljivanja sljedećih značajnih simbola <code> + - * ] [ ? @ </code> u asemblerskom kodu. Za navedene simbole pretpostavljamo da će biti posebno izraženi u kodu jer njihova velika frekvencija općenito znači da je on namjenjen za izbjegavanje otkrivanja od zlonamjernog napada.

<br>

Navedene značajke dobili smo pokretanjem funkcije <code>asm_symbols</code> slanjem varijable <code>asm_code</code>, odnosno .asm datoteke. Kao izlazni podatak, funkcija nam vraća polje od 7 vrijednosti koje reprezentiraju značajke.

In [9]:
def asm_symbols(asm_code):
    symbols = [0]*7
    for row in asm_code:
        if '*' in row:
            symbols[0] += 1
        if '-' in row:
            symbols[1] += 1
        if '+' in row:
            symbols[2] += 1
        if '[' in row:
            symbols[3] += 1
        if ']' in row:
            symbols[4] += 1
        if '@' in row:
            symbols[5] += 1
        if '?' in row:
            symbols[6] += 1

    return symbols

<code> rezultat: 7 značajki </code>
<br><br>

### opcode

<i>Opcode</i> su instrukcije koje specificiraju koje operacije procesor treba učiniti. Npr. - <code> add </code> je instrukcija koja govori procesoru da treba zbrojiti dvije vrijednosti navedene neposredno nakon te instrukcije.
Izmjenama uobičajenih instrukcija može se izbjeći otkrivanje zlonamjernih napada, ali ne i njihovu klasifikaciju.

Značajke smo dobili računajući broj pojavljivanja svakih od prethodno $93$ fiksiranih instrukcija u funkciji <code>asm_opcodes</code>.

In [12]:
def asm_opcodes(asm_code):
    opcodes = ['add','al','bt','call','cdq','cld','cli','cmc','cmp','const','cwd','daa','db'
                ,'dd','dec','dw','endp','ends','faddp','fchs','fdiv','fdivp','fdivr','fild'
                ,'fistp','fld','fstcw','fstcwimul','fstp','fword','fxch','imul','in','inc'
                ,'ins','int','jb','je','jg','jge','jl','jmp','jnb','jno','jnz','jo','jz'
                ,'lea','loope','mov','movzx','mul','near','neg','not','or','out','outs'
                ,'pop','popf','proc','push','pushf','rcl','rcr','rdtsc','rep','ret','retn'
                ,'rol','ror','sal','sar','sbb','scas','setb','setle','setnle','setnz'
                ,'setz','shl','shld','shr','sidt','stc','std','sti','stos','sub','test'
                ,'wait','xchg','xor']
    opcodes_values = [0]*len(opcodes)
    for row in asm_code:
        parts = row.split()

        for opcode in opcodes:
            if opcode in parts:
                opcodes_values[opcodes.index(opcode)] += 1 
                break
    return opcodes_values


<code> rezultat: 93 značajke </code>
<br><br>

### registri

Skupina značajki koja govori o frekvenciji korištenja registara koji nisu namjenjeni za zadatke. Iako je moguće izmjeniti imena registara, ovako dobivene značajke mogu biti korisne u klasifikaciji <i>malware</i>-a u jednu od familija.

Pozivanjem funkcije <code>asm_registers</code> dobivamo nove 26 značajke koje predstavljuju frekvencije korištenja registara <code>edx, esi, es, ...</code> u asemblerskom kodu.

In [13]:
def asm_registers(asm_code):
    registers = ['edx','esi','es','fs','ds','ss','gs','cs','ah','al',
                 'ax','bh','bl','bx','ch','cl','cx','dh','dl','dx',
                 'eax','ebp','ebx','ecx','edi','esp']
    registers_values = [0]*len(registers)
    for row in asm_code:
        parts = row.replace(',',' ').replace('+',' ').replace('*',' ').replace('[',' ').replace(']',' ') \
                    .replace('-',' ').split()
        for register in registers:
            registers_values[registers.index(register)] += parts.count(register)
    return registers_values

<code> rezultat: 26 značajke </code>
<br><br>

### sekcije

Svaki PE (<i>Portable Executable</i>) sastoji se od nekih predefiniranih dijelova - npr <code> .text, .data, .bss, .rdata, .edata, .idata, .rsrc, .tls, .reloc</code>. Na temelju tih dijelova, pozivajući funkciju <code>asm_sections</code>, dobili smo novih 24 značajke rečunajući:
* broj linija u .bss sekciji
* broj linija u .edata sekciji
* broj linija u .idata sekciji
* broj linija u .rdata sekciji
* broj linija u .rsrc sekciji
* broj linija u .text sekciji
* broj linija u .tls sekciji
* broj linija u .reloc sekciji
* ukupan broj sekcija 
* ukupan broj nepoznatih sekcija
* ukupan broj linija u nepoznatim sekcijama
* razmjer linija poznatih sekcija u odnosu na ukupan broj linija u datoteci 
* razmjer linija nepoznatih sekcija u odnosu na ukupan broj linija u datoteci 
* razmjer nepoznatih sekcija u odnosu na cijelu datoteku
* razmjer linija u .bss sekciji u odnosu na ukupan broj linija u datoteci
* razmjer linija u .edata sekciji u odnosu na ukupan broj linija u datoteci
* razmjer linija u .idata sekciji u odnosu na ukupan broj linija u datoteci
* razmjer linija u .rdata sekciji u odnosu na ukupan broj linija u datoteci
* razmjer linija u .rsrc sekciji u odnosu na ukupan broj linija u datoteci
* razmjer linija u .text sekciji u odnosu na ukupan broj linija u datoteci
* razmjer linija u .tls sekciji u odnosu na ukupan broj linija u datoteci
* razmjer linija u .reloc sekciji u odnosu na ukupan broj linija u datoteci

In [16]:
def asm_sections(asm_code):
    section_names = []
    for row in asm_code:
        section_name = [row[0:np.core.defchararray.index(row, ':')]]
        if section_name != 'HEADER':
            section_names += section_name

    known_sections = ['.text', '.data', '.bss', '.rdata', '.edata', '.idata', '.rsrc', '.tls', '.reloc']
    sections_values = [0]*24
    unknown_sections = []
    unknown_lines = 0
    number_of_sections = len(section_names)

    for section in section_names:

        if section in known_sections:
            section_index = known_sections.index(section)
            sections_values[section_index] += 1
        else:
            unknown_sections.append(section)
            unknown_lines += 1

    uni_section_names_len = len(np.unique(section_names))
    uni_unknown_section_names_len = len(np.unique(unknown_sections))
    uni_known_section_names_len = 0
    for i in range(0,8):
        if sections_values[i] != 0:
            uni_known_section_names_len += 1

    sections_values[9] = uni_section_names_len
    sections_values[10] = uni_unknown_section_names_len
    sections_values[11] = unknown_lines

    for i in range(0,8):
        sections_values[i + 12] = float(sections_values[i])/ number_of_sections

    sections_values[21] = float(uni_known_section_names_len) / uni_section_names_len
    sections_values[22] = float(uni_unknown_section_names_len) / uni_section_names_len
    sections_values[23] = float(unknown_lines) / number_of_sections

    return sections_values, section_names

<code> rezultat: 24 značajke </code>
<br><br>

### ključne riječi

Skupina od $95$ značajki koja objašnjava frekvencije korištenja nekih ključnih riječi - npr. <code>hkey_local_machine</code> koja označava pristup putanji <i>Windows registry</i>-a (tekstna baza podataka koja sadrži manje ili više važne podatke/zapise vezane uz rad Windows operacijskog sustava ).

In [22]:
def asm_misc(asm_code):

    keywords = ['Virtual','Offset','loc','Import','Imports','var','Forwarder','UINT','LONG','BOOL','WORD','BYTES','large',
                'short','dd','db','dw','XREF','ptr','DATA','FUNCTION','extrn','byte','word','dword','char','DWORD','stdcall',
                'arg','locret','asc','align','WinMain','unk','cookie','off','nullsub','DllEntryPoint','System32','dll','CHUNK',
                'BASS','HMENU','DLL','LPWSTR','void','HRESULT','HDC','LRESULT','HANDLE','HWND','LPSTR','int','HLOCAL','FARPROC',
                'ATOM','HMODULE','WPARAM','HGLOBAL','entry','rva','COLLAPSED','config','exe','Software','CurrentVersion',
                '__imp_','INT_PTR','UINT_PTR','---Seperator','PCCTL_CONTEXT','__IMPORT_','INTERNET_STATUS_CALLBACK','.rdata:',
                '.data:','.text:','case','installdir','market','microsoft','policies','proc','scrollwindow','search','trap',
                'visualc','___security_cookie','assume','callvirtualalloc','exportedentry','hardware','hkey_current_user',
                'hkey_local_machine','sp-analysisfailed','unableto']

    keywords_values = [0]*len(keywords)

    for i in range(len(keywords)):
        if keywords[i] in row:
            keywords_values[i] += 1 
            break
    return keywords_values

<code> rezultat: 95 značajki </code>
<br><br>

### API

Značajke koje predstavljuju frekvencije pozivanja svakog od $794$ najkorištenijih API-ja u zlonamjernim datotekama.
Njihove vrijednosti dobili smo pozivanjem funkcije <code>asm_APIs</code>.

<br>

Nažalost, ove značajke su nazadajauće za neke podskupove podataka jer neki primjerci ne sadrže niti jedan poziv API-ja zbog načina pakiranja. Na primjer, uzorak s hash kodom <code>00yCuplj2VTc9ShXZDvnxz</code> je pakiran sa [aspack](http://www.aspack.com/)-om i ne sadrži niti jedan API  poziv te se većina asemblerskog koda sastoji od <i><b>data define</b></i> instrukcija. Upravo zbog toga u skup značajki dodana je još jedna skupina - <b> data define </b>

In [15]:
def asm_APIs(asm_code, apis):
    apis_values = [0]*len(apis)
    for row in asm_code:
        for i in range(len(apis)):
            if apis[i] in row:
                apis_values[i] += 1 
                break
    return apis_values

<code> rezultat: 794 značajke </code>
<br><br>

### data define

$24$ značajke dobivene računanjem sljedećih vrijednosti:
* razmjer korištenja instrukcije <code>db</code> u cijeloj datoteci
* razmjer korištenja instrukcije <code>dd</code> u cijeloj datoteci
* razmjer korištenja instrukcije <code>dw</code> u cijeloj datoteci
* razmjer korištenja instrukcija <code>db, dd, dw</code> u cijeloj datoteci
* razmjer korištenja instrukcije <code>db</code> sa $0$ parametara u cijeloj datoteci
* razmjer korištenja instrukcije <code>db</code> sa ne $0$ parametara u cijeloj datoteci
* razmjer korištenja instrukcije <code>dd</code> u .text sekciji 
* razmjer korištenja instrukcije <code>db</code> u .text sekciji 
* razmjer korištenja instrukcije <code>dd</code> u .rdata sekciji 
* razmjer korištenja instrukcije <code>db</code> sa jednim ne $0$ parametrom u .rdata sekciji
* razmjer korištenja instrukcije <code>db</code> sa jednim ne $0$ parametrom u .data sekciji
* razmjer korištenja instrukcije <code>db</code> sa jednim ne $0$ parametrom u cijeloj datoteci
* razmjer korištenja instrukcije <code>dd</code> sa $4$ parametara 
* razmjer korištenja instrukcije <code>dd</code> sa $5$ parametara 
* razmjer korištenja instrukcije <code>dd</code> sa $6$ parametara 
* razmjer korištenja instrukcije <code>dd</code> sa $4$ parametara u cijeloj datoteci
* razmjer korištenja instrukcije <code>dd</code> sa $5$ parametara u cijeloj datoteci
* razmjer korištenja instrukcije <code>dd</code> sa $6$ parametara u cijeloj datoteci
* razmjer korištenja instrukcije <code>db</code> sa jednim ne $0$ parametrom u .idata sekciji
* razmjer korištenja instrukcije <code>db</code> sa jednim ne $0$ parametrom u nepoznatim sekcijama
* razmjer korištenja instrukcije <code>dd</code> sa $4$ parametara u nepoznatim sekcijama
* razmjer korištenja instrukcije <code>dd</code> sa $5$ parametara u nepoznatim sekcijama
* razmjer korištenja instrukcije <code>dd</code> sa $6$ parametara u nepoznatim sekcijama
* razmjer korištenja instrukcije <code>db</code> sa $0$ parametara u odnosu na broj korištenja te instrukcije sa ne $0$ parametara

In [19]:
def asm_data_define(asm_code):
    #Allcounter  = 0
    dbCounter = 0
    ddCounter = 0
    dwCounter = 0
    dcCounter = 0
    db0Counter = 0
    dbN0Counter = 0

    all = 0
    text = 0
    rdata = 0
    data = 0
    dd_text = 0
    db_text = 0
    dd_rdata = 0
    db3_rdata = 0
    db3_data = 0
    db3_all = 0
    dd4 = 0
    dd5 = 0
    dd6 = 0

    idata = 0
    # NotdataNottext
    NdNt = 0

    db3_idata = 0
    db3_text = 0
    db3_rsrc = 0
    db3_NdNt = 0
    db3_all = 0
    db3_zero = 0

    dd_text = 0
    db_text = 0
    dd_rdata = 0
    db3_data = 0
    db3_all = 0
    dd4_NdNt = 0
    dd5_NdNt = 0
    dd6_NdNt = 0

    for row in asm_code:
        RowItems = row.split()
        Section = row.split(':')[0]
        RowComma = row.split(',')

        all += 1
        dbCounter += RowItems.count('db')
        ddCounter += RowItems.count('dd')
        dwCounter += RowItems.count('dw')
        if len(RowItems)>3:
            if RowItems[1]=='00' and RowItems[2]=='db':
                db0Counter += 1

        if Section=='.text':
            text +=1
            dd_text += RowItems.count('dd')
            db_text += RowItems.count('db')
        elif Section=='.rdata':
            rdata +=1
            dd_rdata += RowItems.count('dd')
            if len(RowItems) == 4 or len(RowItems) == 6:
                if RowItems[2] == 'db':
                    db3_rdata += RowItems.count('db')
        elif Section=='.data':
            data +=1
            if len(RowItems) == 4 or len(RowItems) == 6:
                if RowItems[2] == 'db':
                    db3_data += RowItems.count('db')

        if len(RowItems) == 4 or len(RowItems) == 6:
                if RowItems[2] == 'db':
                    db3_all += RowItems.count('db')
        elif Section=='.idata':
            idata +=1
            if len(RowItems) == 4 or len(RowItems) == 6:
                if RowItems[2] == 'db':
                    db3_idata += 1
        else:
            NdNt += 1
            if len(RowItems) == 4 or len(RowItems) == 6:
                if RowItems[2] == 'db':
                    db3_NdNt += 1

            if len(RowComma)==4:
                dd4_NdNt += RowItems.count('dd')

            if len(RowComma)==5:
                dd5_NdNt += RowItems.count('dd')

            if len(RowComma)==6:
                dd6_NdNt += RowItems.count('dd')

        if len(RowComma)==4:
            dd4 += RowItems.count('dd')

        if len(RowComma)==5:
            dd5 += RowItems.count('dd')

        if len(RowComma)==6:
            dd6 += RowItems.count('dd')

        if len(RowItems) == 4 or len(RowItems) == 6:
            if RowItems[2] == 'db':
                db3_all += 1
                if RowItems[1] == '00':
                    db3_zero += 1


    dcCounter = dbCounter + ddCounter + dwCounter
    db_por = float(dbCounter)/all
    dd_por = float(ddCounter)/all
    dw_por = float(dwCounter)/all
    dc_por = float(dcCounter)/all
    db0_por = dbN0_por = 0
    if dbCounter!=0:
        db0_por = float(db0Counter)/dbCounter
        dbN0_por = float(dbCounter - db0Counter)/dbCounter

    ############################

    Res_dd_text = 0
    Res_db_text = 0
    Res_dd_rdata = 0
    Res_db3_rdata = 0
    Res_db3_data = 0

    if text!=0:
        Res_dd_text = float(dd_text)/text
        Res_db_text = float(db_text)/text

    if rdata!=0:
        Res_dd_rdata = float(dd_rdata)/rdata
        Res_db3_rdata = float(db3_rdata)/rdata

    if data!=0:
        Res_db3_data = float(db3_data)/data

    Res_db3_all = float(db3_all)/all

    Res_dd4_all = float(dd4)/all
    Res_dd5_all = float(dd5)/all
    Res_dd6_all = float(dd6)/all

    Output = [Res_dd_text,Res_db_text,Res_dd_rdata,Res_db3_rdata \
              , Res_db3_data, Res_db3_all, dd4, dd5, dd6 \
              , Res_dd4_all, Res_dd5_all, Res_dd6_all]


    Res_db3_idata = 0
    Res_db3_NdNt = 0
    Res_dd4_NdNt = 0
    Res_dd5_NdNt = 0
    Res_dd6_NdNt = 0
    Res_db3_all_zero = 0

    if idata!=0:
        Res_db3_idata = float(db3_idata)/idata

    if NdNt!=0:
        Res_db3_NdNt = float(db3_NdNt)/NdNt
        Res_dd4_NdNt = float(dd4_NdNt)/NdNt
        Res_dd5_NdNt = float(dd5_NdNt)/NdNt
        Res_dd6_NdNt = float(dd6_NdNt)/NdNt

    if db3_all!=0:
        Res_db3_all_zero = float(db3_zero)/db3_all

    Output2 = [Res_db3_idata, Res_db3_NdNt, Res_dd4_NdNt \
              , Res_dd5_NdNt, Res_dd6_NdNt, Res_db3_all_zero ]


    return [db_por, dd_por, dw_por, dc_por, db0_por, dbN0_por] + Output + Output2

<code> rezultat: 24 značajki </code>
<br><br>